In [117]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [6]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess    

def tidy_sentence(sentence, vocabulary):
    return [word for word in simple_preprocess(sentence) if word in vocabulary]    

def compute_sentence_similarity(sentence_1, sentence_2, model_wv):
    vocabulary = set(model_wv.index_to_key)    
    tokens_1 = tidy_sentence(sentence_1, vocabulary)    
    tokens_2 = tidy_sentence(sentence_2, vocabulary)    
    return model_wv.n_similarity(tokens_1, tokens_2)

sim = compute_sentence_similarity('this is a sentence', 'this is also a sentence', model)
print(sim)

0.9596676


### Get Word2Vec vectors forgiven sentence:

In [109]:
sentence = tidy_sentence("Going back to school is not identical with giv...", set(model.index_to_key))
np.mean([model[word] for word in sentence], axis=0).shape,  # to average vectors

((300,),)

In [111]:
np.mean([model[word] for word in sentence], axis=0);

generate vectors for first 1000 comments

In [124]:
def save_sentence_vector(row, np_array):
    sentence = tidy_sentence(row['comment'], set(model.index_to_key))
    w2v_vectors[row.name] = np.mean([model[word] for word in sentence], axis=0)

Слишком медленно :(

In [125]:
w2v_vectors = np.zeros([1000, 300])
tqdm.pandas()
train.progress_apply(save_sentence_vector, np_array=w2v_vectors, axis=1)

  0%|          | 235/440535 [01:53<58:49:55,  2.08it/s]


KeyboardInterrupt: 

In [10]:
train = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_train.jsonl', lines=True)

In [11]:
train = train.explode('comments')
train['score'] = train['comments'].str['score']
train['comment'] = train['comments'].str['text']
train = train.drop(columns=['comments'])

In [63]:
train.head(20)

,text,score,comment
0,How many summer Y Combinator fundees decided n...,0,Going back to school is not identical with giv...
0,How many summer Y Combinator fundees decided n...,1,There will invariably be those who don't see t...
0,How many summer Y Combinator fundees decided n...,2,For me school is a way to be connected to what...
0,How many summer Y Combinator fundees decided n...,3,I guess it really depends on how hungry you ar...
0,How many summer Y Combinator fundees decided n...,4,I know pollground decided to go back to school...
1,CBS acquires last.fm for $280m,0,It will be curious to see where this heads in ...
1,CBS acquires last.fm for $280m,1,Does this mean that there's now a big-name com...
1,CBS acquires last.fm for $280m,2,Also on BBC News: http://news.bbc.co.uk/1/low...
1,CBS acquires last.fm for $280m,3,I don't understand what they do that is worth ...
1,CBS acquires last.fm for $280m,4,sold out too cheaply. given their leadership p...


In [66]:
first  = str(list(train.query('score == 0').head(20).values[2]))
second  = str(list(train.query('score == 4').head(20).values[2]))

compute_sentence_similarity(first, second ,model)

0.79187745

In [74]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 0').head(100).values[i]))
    second  = str(list(train.query('score == 4').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

0.7469575899839401

In [75]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 0').head(100).values[i]))
    second  = str(list(train.query('score == 0').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

0.7909232354164124

In [76]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 1').head(100).values[i]))
    second  = str(list(train.query('score == 1').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

0.7530625116825104

In [77]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 2').head(100).values[i]))
    second  = str(list(train.query('score == 2').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

0.7392598530650138

In [78]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 3').head(100).values[i]))
    second  = str(list(train.query('score == 3').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

0.7085300156474114

tfidf

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = train['comment'].head(100000).values
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(corpus)
print(X.shape)

(100000, 5000)


from: https://stackoverflow.com/questions/53294482/how-to-get-tf-idf-scores-for-the-words

In [99]:
tfidf_dict = dict(zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0)))
# sort in descending order
tfidf_dict = dict(sorted(tfidf_dict.items(), key=lambda x: x[1], reverse=True))

In [100]:
# initialise dataframe
tfidf_df = pd.DataFrame.from_dict(tfidf_dict, orient='index', columns=['tfidf'])
# name the index
tfidf_df.index = tfidf_df.index.rename('token')
# display first 5 rows
tfidf_df.head()

,tfidf
token,
the,9042.799581
to,6964.571971
of,5309.919585
and,5196.428679
it,4883.370793


In [102]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
svd = TruncatedSVD(n_components=256, n_iter=7, random_state=42)
svd.fit(X)
# print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())

0.32680192262455676
